In [1]:
import torch
from pointnet.model import PointNet

state = torch.load("logs/8192 octants l=4 rgb/model_weights_550.pt")
l = int(state["coef_mean"].shape[0]**0.5)-1
batch_norm = "bn1.weight" in state
model = PointNet(l,3,batch_norm=batch_norm)
model.load_state_dict(state)
model.eval()


traced_model = torch.jit.trace_module(model,{
    "forward":[torch.rand((1,3)),torch.rand((1,3)),torch.zeros(1,dtype=torch.long)]
})

torch.jit.save(traced_model,"traced_model_cpu.pt")

traced_model = torch.jit.trace_module(model.cuda(),{
    "forward":[torch.rand((1,3)).cuda(),torch.rand((1,3)).cuda(),torch.zeros(1,dtype=torch.long).cuda()]
})

torch.jit.save(traced_model,"traced_model_gpu.pt")

In [ ]:
traced_model.cuda()(torch.rand((1,3)).cuda(),torch.rand((1,3)).cuda(),torch.zeros(1,dtype=torch.long).cuda())

In [ ]:
for b in traced_model.buffers():
    print(b.device)

In [ ]:
from pointnet.dataset import OctantDataset

ds = OctantDataset("datasets/neuschwanstein/octants_1024max_sh",preload=True)

ds

In [ ]:
from pointnet.model import PointNet
import torch

model = PointNet(10,4,batch_norm=True,use_dropout=True)
model.eval()
model.load_state_dict(torch.load("logs/100k top_variance, with_alpha, camera_loss only, small dropout/model.pt"))

traced_model = torch.jit.trace_module(model,{
    "forward":[torch.rand((1,3)),torch.rand((1,4)),torch.zeros(1,dtype=torch.long)]
})

torch.jit.save(traced_model,"traced_model.pt")

In [ ]:
model =torch.jit.load("logs/100k top_variance l=5/traced_model.pt").cuda()
model.eval()

In [ ]:
model(torch.rand((1,3)).cuda(),torch.rand((1,4)).cuda(),torch.zeros(1,dtype=torch.long).cuda())

In [ ]:
from tqdm import tqdm
from pointnet.pointclouds import collate_batched

with open("datasets/neuschwanstein/octants_1024max_sh/coefs.raw","wb") as f:
    batches = torch.arange(len(ds)).split(256)
    for batch in tqdm(batches):
        pc = collate_batched([ds[i][0] for i in batch])
        coefs = model(pc.points_packed().cuda(),pc.features_packed().cuda(),pc.packed_to_cloud_idx().cuda())
        octant_id = int(pc.file_names[0].split("/")[-1].split(".")[0].split("_")[1])
        f.write(octant_id.to_bytes(8,"little"))
        f.write(coefs.detach().flatten().cpu().numpy().tobytes())

In [ ]:
x = torch.ones(125,3)
torch.cat([x,x],dim=0).shape